In [2]:
library(tidyverse)

── Attaching pa

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [84]:
con = DBI::dbConnect(RMariaDB::MariaDB(), groups='mariaDB')

In [4]:
carly_orfs = DBI::dbGetQuery(con, "select * from CARVUNIS_YEAST.carly_orfs a
LEFT JOIN CARVUNIS_YEAST.orf_ids_crossmap b
ON a.orf_id=b.orf_id
LEFT JOIN aaron.scer_orfs_translation_info c
ON b.aaron_orfid=c.orf_id
LEFT JOIN (SELECT orf_id, high_information FROM aaron.scer_orfs_translation_info_100722) d
ON c.orf_id=d.orf_id
where a.chromosome!='chrmt' AND c.orf_class!='orf_class'")%>%distinct()

In [5]:
blastp_all_by_all = data.table::fread("/home/oma21/coexpression/data/interim/blast_files/blast_results/s288c_blastp/scer_all_orfs_by_all_orfs.blast")%>%separate(V1, sep="::", into = c("orf_id",'coords'))%>%
    separate(V2, sep="::", into = c("hit_orf_id",'hit_coords'))%>%filter(orf_id!=hit_orf_id)

In [6]:
blastp_vs_yeasts_files = list.files("/home/acwach/YeastTest/blastp")

In [7]:
blastp_vs_yeasts = map(blastp_vs_yeasts_files, function(x) data.table::fread(paste0("/home/acwach/YeastTest/blastp/",x))%>%mutate(filename=x))%>%data.table::rbindlist()

In [8]:
blastp_vs_yeasts = blastp_vs_yeasts%>%separate(V1,c('chr','start','end','strand'), sep='_', convert=T)%>%
    mutate(chr=chr+1, start=start+1, end=end+1, strand = ifelse(strand==0, '+','-'),
          orf_coord_id = paste0(chr,'_',start,'_',end,'_',strand))%>%
    group_by(orf_coord_id)%>%filter(V11==min(V11))%>%ungroup()%>%distinct(orf_coord_id, V11)

In [9]:
carly_orfs%>%select(orf_class)%>%table

.
                  Dubious                      None                pseudogene 
                      676                    260359                         9 
transposable_element_gene           Uncharacterized                  Verified 
                       47                       698                      4922 

In [16]:
aaron_table = DBI::dbGetQuery(con, "select * from aaron.scer_orfs_translation_info")

In [17]:
colnames(aaron_table)

[1] "orf_id"                   "chr"                     
 [3] "first_coord"              "second_coord"            
 [5] "strand"                   "gene_systematic_name"    
 [7] "orf_class"                "length"                  
 [9] "rfc"                      "coding_score"            
[11] "scer_gene_eval"           "distant_homolog_evalue"  
[13] "is_intergenic"            "evolutionary_class"      
[15] "up_extend_gene"           "is_candidate"            
[17] "pval"                     "qval"                    
[19] "ribo_reads"               "chx_plus_pval"           
[21] "chx_minus_pval"           "ypd_media_pval"          
[23] "sd_media_pval"            "ypd_chx_minus_pval"      
[25] "chx_plus_reads"           "chx_minus_reads"         
[27] "ypd_media_reads"          "sd_media_reads"          
[29] "ypd_chx_minus_reads"      "sense_verified_overlap"  
[31] "sense_unchar_overlap"     "sense_dubious_overlap"   
[33] "sense_pseudogene_overlap" "sense_te_overlap"        
[35] "sense_blocked_overlap"    "anti_verified_overlap"   
[37] "anti_unchar_overlap"      "anti_dubious_overlap"    
[39] "anti_pseudogene_overlap"  "anti_te_overlap"         
[41] "anti_blocked_overlap"     "tRNA_overlap"            
[43] "ltr_overlap"              "snoRNA_overlap"          
[45] "ars_overlap"              "sn_overlap"              
[47] "ncRNA_overlap"            "neighbor1"               
[49] " neighbor1_dist"          "neighbor2"               
[51] "neighbor2_dist"           "start_codon_overlap"     
[53] "stop_codon_overlap"       "orf_coord_id"            
[55] "use_blast0"               "use_blast1"              
[57] "use_blast2"               "use_blast3"              
[59] "use_blast4"               "use_blast5"              
[61] "use_blast6"               "use_blast7"              
[63] "good_homology1"           "good_homology2"          
[65] "good_homology3"           "good_homology4"          
[67] "good_homology5"           "good_homology6"          
[69] "good_homology7"           "high_information"        
[71] "num_orthologs"            "rfc0"                    
[73] "rfc1"                     "rfc2"                    
[75] "rfc3"                     "rfc4"                    
[77] "rfc5"                     "rfc6"

In [22]:
aaron_denovo_parsimony =aaron_table%>%rename(rfc_spar = rfc0, 
                                                                                                    rfc_smik = rfc1,
                                                                                                    rfc_sjur = rfc2,
                                                                                                    rfc_skud = rfc3,
                                                                                                    rfc_sarb = rfc4,
                                                                                                    rfc_sbay = rfc5,
                                                                                                    rfc_seub = rfc6,
                                                                                                    good_homology_spar = good_homology1,
                                                                                                    good_homology_smik = good_homology2,
                                                                                                     good_homology_sjur = good_homology3,
                                                                                                     good_homology_skud = good_homology4,
                                                                                                     good_homology_sarb = good_homology5,
                                                                                                     good_homology_sbay = good_homology6,
                                                                                                     good_homology_seub = good_homology7,
                                                                                                    )%>%
select(-rfc_sjur, -good_homology_sjur)%>%group_by(orf_id)%>%
mutate(rfc_sbay_combined = case_when(good_homology_seub==1 & good_homology_sbay==1 ~ max(rfc_sbay,rfc_seub),
                                     good_homology_seub==1 & good_homology_sbay!=1 ~ rfc_seub,
                                     good_homology_seub!=1 & good_homology_sbay==1 ~ rfc_sbay,
                                     good_homology_seub!=1 & good_homology_sbay!=1 ~ -1
                                 ))%>%ungroup()%>%
    mutate(proven_denovo= case_when(
        rfc_sbay_combined <.6 & rfc_sarb <.6 & rfc_sbay_combined!=-1 & good_homology_sarb==1  ~ TRUE,
        rfc_sbay_combined==-1 & rfc_sarb<0.6 & rfc_skud <0.6 & good_homology_sarb==1 & good_homology_skud==1 ~ TRUE,
        rfc_sbay_combined==-1 &  good_homology_sarb==0 & rfc_skud <0.6 & good_homology_skud==1 & rfc_smik <0.6 & good_homology_smik==1 ~ TRUE,
        rfc_sbay_combined==-1 & good_homology_sarb==0 & good_homology_skud==0 & rfc_smik <0.6 & good_homology_smik==1 & rfc_spar<0.6 & good_homology_spar==1  ~ TRUE,
        TRUE ~ FALSE
    ))%>%
    select(orf_id, proven_denovo)

In [23]:
carly_orfs_with_parsimony = carly_orfs%>%
    left_join(aaron_denovo_parsimony[,c('orf_id','proven_denovo')], by=c('aaron_orfid'='orf_id'))%>%
    mutate(
        transcript = sprintf("chr%s_%i", chr, first_coord),
        gene = case_when(
          gene_systematic_name != "X" ~ gene_systematic_name,
          up_extend_gene != "X" ~ up_extend_gene
        )
      )%>%#left_join(blastp_results_min, by=c('orf_id'='orf_id'))%>%
        left_join(blastp_vs_yeasts, by=c("orf_coord_id"="orf_coord_id"))%>%mutate(distant_homolog_blastp_eval = V11)%>%
        select(-V11) %>%
        mutate(distant_homolog_blastp_eval = ifelse(is.na(distant_homolog_blastp_eval), 100, distant_homolog_blastp_eval))%>%
        left_join(blastp_all_by_all%>%group_by(orf_id)%>%summarise(blastp_against_scer_orfs_eval = min(V11))%>%ungroup()%>%distinct(orf_id,blastp_against_scer_orfs_eval),
                           by=c("orf_id"="orf_id"))%>%mutate(blastp_against_scer_orfs_eval = replace_na(blastp_against_scer_orfs_eval, 100))



In [10]:
# carly_orfs_with_parsimony%>%
#     filter(scer_gene_eval<=1e-2 & orf_coord_id%in%blastn_with_same_coords$orf_coord_id)

In [25]:
carly_orfs_mutated = carly_orfs_with_parsimony%>%
    mutate(scer_gene_eval = ifelse(anti_verified_overlap==1 | anti_unchar_overlap==1 | anti_pseudogene_overlap==1 | anti_dubious_overlap==1 |anti_te_overlap==1 | anti_blocked_overlap==1, 100, scer_gene_eval),
           distant_homolog_evalue = ifelse(anti_verified_overlap==1 | anti_unchar_overlap==1 | anti_pseudogene_overlap==1 | anti_dubious_overlap==1 |anti_te_overlap==1 | anti_blocked_overlap==1, 100, distant_homolog_evalue),
        with_blast_hits_for_denovo = (distant_homolog_evalue<=1e-4 | scer_gene_eval <= 1e-4  | blastp_against_scer_orfs_eval <= 1e-4 | distant_homolog_blastp_eval <= 1e-4),
           with_blast_hits_for_conserved = (distant_homolog_evalue<=1e-4  | distant_homolog_blastp_eval <= 1e-4),
           possibly_denovo = (( with_blast_hits_for_denovo == F ) & proven_denovo==1 & qval<0.05 & is_candidate==1),
           overlaps = rowSums(across(sense_verified_overlap:ncRNA_overlap)),
          possibly_conserved = ((with_blast_hits_for_conserved) | (high_information==1 & rfc >=.8)) & (orf_class=="Verified" | orf_class=='Uncharacterized' | orf_class=='transposable_element_gene') )

In [27]:
overlapping_annotated = DBI::dbGetQuery(con, "select * from omer.annotated_overlap_conservation")

In [28]:
blast_files = list.files("/home/acwach/YeastTest/tblastn_outside/",full.names = T)

In [29]:
blast_extra = data.table::rbindlist(map(blast_files, data.table::fread))

In [30]:
blastp_vs_yeasts_extra_files = list.files("/home/oma21/coexpression/data/interim/blast_files/blast_results/blastp_align/")

In [31]:
blastp_vs_yeasts_extra = map(blastp_vs_yeasts_extra_files, function(x) data.table::fread(paste0("/home/oma21/coexpression/data/interim/blast_files/blast_results/blastp_align/",x))%>%mutate(filename=x))%>%data.table::rbindlist()

In [32]:
blastp_vs_yeasts_extra = blastp_vs_yeasts_extra%>%
    group_by(V1)%>%filter(V11==min(V11))%>%ungroup()%>%distinct(V1, V11)%>%select(gene=V1, V11)

In [33]:
sc_gff = DBI::dbGetQuery(con , "select * from CARVUNIS_YEAST.sc_gff")

In [34]:
gff_genes = sc_gff%>%filter(featuretype=='gene' & chromosome!='chrmt')

In [35]:
gff_genes%>%nrow

[1] 6572

In [36]:
missing_genes = gff_genes%>%filter(Name%in%carly_orfs_mutated$gene==FALSE)

In [37]:
missing_genes_eval = missing_genes%>%left_join(blast_extra%>%group_by(V1)%>%summarise(strongest_hit_eval = min(V11)), by=c('Name'='V1'))%>%
left_join(blastp_vs_yeasts_extra, by=c("Name"="gene"))%>%
mutate(distant_homolog_blastp_eval = V11)%>%
select(-V11)

In [40]:
overlapping_annotated%>%colnames

[1] "Name"                                         
 [2] "orf_classification"                           
 [3] "overlapping_orf_name"                         
 [4] "overlapping_orf_classification"               
 [5] "is_sense_overlap"                             
 [6] "rfc"                                          
 [7] "overlapping_orf_rfc"                          
 [8] "high_information"                             
 [9] "overlapping_orf_high_information"             
[10] "distant_homolog_evalue"                       
[11] "overlapping_orf_distant_homolog_evalue"       
[12] "strongest_nonoverlapping_eval"                
[13] "overlapping_orf_strongest_nonoverlapping_eval"
[14] "rfc_comparison"                               
[15] "nonoverlapping_blast_conservation"            
[16] "is_conserved"

In [41]:
carly_orfs_including_multiple_exons = carly_orfs_mutated%>%
    bind_rows(missing_genes_eval%>%
              mutate(possibly_conserved = (strongest_hit_eval <=1e-4 | 
                     distant_homolog_blastp_eval <= 1e-4) & (orf_classification%in%c("Verified",'Uncharacterized')), 
                     possibly_denovo=F,
                     orf_class=orf_classification))%>%
    left_join(overlapping_annotated%>%
              select(Name,is_conserved_with_overlap=is_conserved) %>%
              distinct(), by=c("gene"="Name"))%>%
    mutate(possibly_conserved = ifelse(is.na(is_conserved_with_overlap)==F & 
                                       distant_homolog_blastp_eval > 1e-4 & (orf_classification%in%c("Verified","Uncharacterized")), 
                                       is_conserved_with_overlap==1,
                                       possibly_conserved),
          possibly_denovo = ifelse(possibly_conserved, F, possibly_denovo))

    #drop_na(is_conserved_with_overlap)%>%filter(possibly_conserved!=is_conserved_with_overlap)


In [42]:
carly_orfs_including_multiple_exons%>%
select(possibly_conserved,possibly_denovo)%>%
table


                  possibly_denovo
possibly_conserved  FALSE   TRUE
             FALSE 252661   8638
             TRUE    5688      0

In [43]:
coexpression_orf_list <- carly_orfs_including_multiple_exons%>%
    mutate(classification=case_when(
        possibly_denovo ~ 'denovo',
        possibly_conserved ~ 'conserved',
        possibly_denovo==FALSE & possibly_conserved==FALSE & orf_class=='None' & qval >0.5 & is_candidate==1~ 'intergenic' ,
        TRUE ~ 'other'
    ))

In [44]:
coexpression_orf_list$classification%>%table

.
 conserved     denovo intergenic      other 
      5688       8638     109244     143417 

In [539]:
coexpression_orf_list$classification%>%table

.
 conserved     denovo intergenic      other 
      5877       8636     109244     143230 

In [46]:
143230-143417

[1] -187

In [ ]:
coexpression_orf_list%>%select(orf_class,classification)%>%table

                           classification
orf_class                   conserved denovo intergenic  other
  Dubious                           0     85          0    595
  None                              0   8461     109244 142654
  pseudogene                        0      0          0      9
  transposable_element_gene        47      0          0      0
  Uncharacterized                 495     88          0    142
  Verified                       5146      4          0     17

In [540]:
coexpression_orf_list%>%select(orf_class,classification)%>%table

                           classification
orf_class                   conserved denovo intergenic  other
  Dubious                         181     84          0    415
  None                              0   8460     109244 142655
  pseudogene                        8      0          0      1
  transposable_element_gene        47      0          0      0
  Uncharacterized                 494     88          0    143
  Verified                       5147      4          0     16

In [557]:
# coexpression_orf_list%>%filter(gene=='YCL058C')%>%select(gene,rfc,high_information, distant_homolog_evalue, scer_gene_eval, proven_denovo,classification, orf_coord_id)

In [482]:
# coexpression_orf_list%>%filter(gene=='YBR196C-A')%>%select(gene,rfc,high_information, distant_homolog_evalue, scer_gene_eval, proven_denovo,classification)

In [484]:
# coexpression_orf_list%>%filter(gene=='YNL269W')%>%select(gene,rfc,high_information, distant_homolog_evalue, scer_gene_eval, proven_denovo,classification, is_candidate,qval)

In [567]:
# coexpression_orf_list%>%filter(gene=='YGL168W')%>%select(gene,rfc,high_information, distant_homolog_evalue, scer_gene_eval, proven_denovo,classification)

In [49]:
coexpression_orf_list_upload<- coexpression_orf_list %>%
    mutate(chr_num = str_replace(chromosome, 'chr','')%>%gtools::roman2int(),
        transcript = glue::glue("chr{chr_num}_{coor1}"),
        transcript_coor2 = glue::glue("chr{chr_num}_{coor2}"),
        orf_coord_id = glue::glue("chr{chr_num}_{coor1}_{coor2}_{strand}"),
           length = abs(coor2-coor1)+1,
          gene = ifelse(is.na(aaron_orfid)==FALSE, gene, Name))


In [50]:
denovo= coexpression_orf_list_upload%>%filter(classification=='denovo')%>%select(chromosome, coor1, coor2, orf_id, rfc, strand)#%>%write_tsv('../foo_denovo.bed', col_names = FALSE)

In [51]:
conserved = coexpression_orf_list_upload%>%filter(classification=='conserved')%>%select(chromosome, coor1, coor2, orf_id, rfc, strand)#%>%write_tsv('../foo_conserved.bed', col_names = FALSE)

In [52]:
overlap_list = list()
for(i in 1:nrow(denovo)){
    if(i%%1000==0){
        print(i)    
    }
    
    res = conserved%>%filter(chromosome == denovo$chromosome[i], 
                       (coor1< denovo$coor1[i] & coor2 > denovo$coor1[i]) | 
                      (coor1< denovo$coor2[i] & coor2 > denovo$coor2[i]) |
                      (coor1<denovo$coor2[i] & coor2 > denovo$coor1[i]), strand==denovo$strand[i])%>%select(orf_id)%>%pull()
    if(length(res)>0){
        overlap_list[[denovo$orf_id[i]]]=res
    }
    
}
coexpression_orf_list_upload<-coexpression_orf_list_upload%>%mutate(classification = ifelse(orf_id%in%names(overlap_list), 'other', classification))

[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000


In [53]:
stop_position_same <- coexpression_orf_list_upload %>%
    group_by(transcript_coor2)%>%filter(n()>1)

In [54]:
stop_position_same_to_remove = stop_position_same %>%
select(transcript, transcript_coor2, gene, Name,classification, orf_coord_id,length)%>%
filter(length==min(length))%>%ungroup()%>%select(orf_coord_id)%>%pull()

In [55]:
start_position_same <- coexpression_orf_list_upload%>%group_by(transcript)%>%filter(n()>1)#%>%filter(gene%in%missing_genes$Name)

In [56]:
start_position_same_to_remove <- start_position_same%>%
select(transcript, transcript_coor2, gene, Name,classification, orf_coord_id,length)%>%
filter(length==min(length))%>%
ungroup()%>%
select(orf_coord_id)%>%pull()

In [61]:
coexpression_orf_list_upload$is_canonical = ifelse(coexpression_orf_list_upload$orf_class=='Verified' | coexpression_orf_list_upload$orf_class=='Uncharacterized' |coexpression_orf_list_upload$orf_class=='transposable_element_gene',
                                                  'canonical','noncanonical')

In [67]:
coexpression_orf_list_upload_final<-coexpression_orf_list_upload%>%filter(orf_coord_id%in%c(start_position_same_to_remove,stop_position_same_to_remove)==F)


In [85]:
coexpression_orf_list_upload_final%>%
filter(is_canonical=='canonical' | qval<0.05)%>%
#mutate(classification = ifelse(orf_id %in% names(overlap_list), "other", classification))%>%
    select(orf_id, chromosome , coor1, coor2, strand, chr_num, aaron_orfid, orf_coord_id, classification,is_canonical, gene,transcript, orf_class, qval, pval)%>%
     DBI::dbWriteTable(con,'coexpressionOrfList_blaste4',., overwrite=T)

In [87]:
coexpression_orf_list_upload_final$classification%>%table

.
 conserved     denovo intergenic      other 
      5687       8625     109244     142936 

In [595]:
coexpression_orf_list_upload$classification%>%table

.
 conserved     denovo intergenic      other 
      5877       8616     109244     143250 

In [70]:
coexpression_orf_list_upload_final%>%select(orf_class,classification)%>%table

                           classification
orf_class                   conserved denovo intergenic  other
  Dubious                           0     85          0    593
  None                              0   8448     109244 142175
  pseudogene                        0      0          0      9
  transposable_element_gene        47      0          0      0
  Uncharacterized                 494     88          0    142
  Verified                       5146      4          0     17

In [71]:
coexpression_orf_list_upload_final%>%select(orf_class,classification,is_canonical)%>%table

, , is_canonical = canonical

                           classification
orf_class                   conserved denovo intergenic  other
  Dubious                           0      0          0      0
  None                              0      0          0      0
  pseudogene                        0      0          0      0
  transposable_element_gene        47      0          0      0
  Uncharacterized                 494     88          0    142
  Verified                       5146      4          0     17

, , is_canonical = noncanonical

                           classification
orf_class                   conserved denovo intergenic  other
  Dubious                           0     85          0    593
  None                              0   8448     109244 142175
  pseudogene                        0      0          0      9
  transposable_element_gene         0      0          0      0
  Uncharacterized                   0      0          0      0
  Verified                       

In [75]:
coexpression_orf_list_upload_final%>%filter(classification=='conserved' | qval<0.05)%>%select(orf_class,classification)%>%table

                           classification
orf_class                   conserved denovo other
  Dubious                           0     85   145
  None                              0   8448 11847
  pseudogene                        0      0     5
  transposable_element_gene        47      0     0
  Uncharacterized                 494     88    28
  Verified                       5146      4     5

In [79]:
coexpression_orf_list_upload_final%>%filter(is_canonical=='canonical' | qval<0.05)%>%select(orf_class,classification)%>%table

                           classification
orf_class                   conserved denovo other
  Dubious                           0     85   145
  None                              0   8448 11847
  pseudogene                        0      0     5
  transposable_element_gene        47      0     0
  Uncharacterized                 494     88   142
  Verified                       5146      4    17

In [76]:
coexpression_orf_list_upload_final%>%filter(qval<0.05)%>%select(orf_class,classification)%>%table

                           classification
orf_class                   conserved denovo other
  Dubious                           0     85   145
  None                              0   8448 11847
  pseudogene                        0      0     5
  transposable_element_gene        20      0     0
  Uncharacterized                 414     88    28
  Verified                       4855      4     5

In [568]:
system("jupyter nbconvert 42-get-gene-list-categories.ipynb --to html_toc --ExtractOutputPreprocessor.enabled=False")